In [1]:
print("hbd")

hbd


In [2]:
!mkdir -p ~/.kaggle
!mv ~/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: rename /Users/nhz/kaggle.json to /Users/nhz/.kaggle/kaggle.json: No such file or directory
chmod: /Users/nhz/.kaggle/kaggle.json: No such file or directory


In [6]:
mkdir -p ~/.kaggle
mv /Users/nhz/Desktop/deepfake/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json


SyntaxError: invalid syntax (1830218467.py, line 1)

In [5]:
!mv ~/Users/nhz/Desktop/deepfake/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: rename /Users/nhz/Users/nhz/Desktop/deepfake/kaggle.json to /Users/nhz/.kaggle/kaggle.json: No such file or directory
chmod: /Users/nhz/.kaggle/kaggle.json: No such file or directory


In [ ]:
# Fix the distutils issue for Python 3.12
import sys
!{sys.executable} -m pip install setuptools --upgrade

# Create a distutils compatibility layer if needed
try:
    import distutils.util
    print("✅ distutils is available!")
except ImportError:
    print("Installing distutils compatibility...")
    import sys
    import types
    
    # Create a mock distutils module
    distutils = types.ModuleType('distutils')
    distutils.util = types.ModuleType('distutils.util')
    
    # Add to sys.modules
    sys.modules['distutils'] = distutils
    sys.modules['distutils.util'] = distutils.util
    
    print("✅ distutils compatibility layer created!")

# Also install any missing dependencies
!{sys.executable} -m pip install --upgrade pip wheel
!{sys.executable} -m pip install --upgrade tensorflow keras opencv-python numpy pillow matplotlib scikit-learn


In [ ]:
# Test TensorFlow imports after fixing distutils
import os
import sys

# Force reload of tensorflow if it was previously imported with errors
if 'tensorflow' in sys.modules:
    del sys.modules['tensorflow']

try:
    print("Attempting to import TensorFlow...")
    import tensorflow as tf
    print("✅ TensorFlow imported successfully!")
    print(f"TensorFlow version: {tf.__version__}")
    
    # Test specific imports that were failing
    from tensorflow.keras.utils import img_to_array
    from tensorflow.keras.applications.inception_v3 import preprocess_input
    print("✅ All TensorFlow Keras imports successful!")
    
    # Test basic functionality
    print("Testing TensorFlow functionality...")
    x = tf.constant([1, 2, 3, 4])
    print(f"✅ TensorFlow is working! Test tensor: {x}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Trying alternative approach...")
    
    # Alternative: Try installing specific TensorFlow version
    !{sys.executable} -m pip install tensorflow==2.15.0 --upgrade
    try:
        import tensorflow as tf
        print("✅ TensorFlow 2.15.0 imported successfully!")
    except Exception as e2:
        print(f"❌ Still failed: {e2}")


In [7]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance
import shutil


In [8]:
train_frame_folder = 'deepfake-detection-challenge/train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)

In [9]:
for key, value in data.items():
    print(data[key])

{'label': 'FAKE', 'split': 'train', 'original': 'vudstovrck.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'jdubbvfswz.mp4'}
{'label': 'REAL', 'split': 'train', 'original': None}
{'label': 'FAKE', 'split': 'train', 'original': 'atvmxvwyns.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'kbvibjhfzo.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'ccfoszqabv.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'fjlyaizcwc.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'ffcwhpnpuw.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'slwkmefgde.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'fysyrqfguw.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'qjlhemtkxk.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'dlpoieqvfb.mp4'}
{'label': 'REAL', 'split': 'train', 'original': None}
{'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}
{'label': 'FAKE', 'split': 'train'

In [10]:
fake_count=0
real_count=0

for key, value in data.items():
    if "FAKE" == data[key]['label']:
        fake_count+=1
    if "REAL" == data[key]['label']:
        real_count+=1
print("Real Videoos:: ", real_count)
print("Fake Videos:: ", fake_count)

Real Videoos::  77
Fake Videos::  323


In [15]:
import os
import cv2
import json
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import dlib

# Constants
IMG_SIZE = (299, 299)  # Resize detected faces to this size
MOTION_THRESHOLD = 20  # Lowered threshold for detecting motion between frames
FRAME_SKIP = 2        # Reduced frame skip to ensure more frames are processed

# Initialize Dlib's frontal face detector (can switch to Haar cascades, MTCNN, or other)
detector = dlib.get_frontal_face_detector()

def extract_faces_from_frame(frame, detector):
    """
    Detects faces in a frame and returns the resized faces.

    Parameters:
    - frame: The video frame to process.
    - detector: Dlib face detector.

    Returns:
    - resized_faces (list): List of resized faces detected in the frame.
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_frame)
    resized_faces = []

    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        crop_img = frame[y1:y2, x1:x2]
        if crop_img.size != 0:  # Ensure cropped image is valid
            resized_face = cv2.resize(crop_img, IMG_SIZE)
            resized_faces.append(resized_face)

    # Debug: Log the number of faces detected
    #print(f"Detected {len(resized_faces)} faces in current frame")
    return resized_faces

def process_frame(video_path, detector, frame_skip):
    """
    Processes frames to extract motion and face data concurrently.

    Parameters:
    - cap: OpenCV VideoCapture object.
    - detector: Dlib face detector.
    - frame_skip (int): Number of frames to skip for processing.

    Returns:
    - motion_frames (list): List of motion-based face images.
    - all_faces (list): List of all detected faces for fallback.
    """
    prev_frame = None
    frame_count = 0
    motion_frames = []
    all_faces = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames to improve processing speed
        if frame_count % frame_skip != 0:
            frame_count += 1
            continue

        # Debug: Log frame number being processed
        #print(f"Processing frame {frame_count}")

        # # Resize frame to reduce processing time (optional, adjust size as needed)
        # frame = cv2.resize(frame, (640, 360))

        # Extract faces from the current frame
        faces = extract_faces_from_frame(frame, detector)
        all_faces.extend(faces)  # Store all faces detected, including non-motion

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is None:
            prev_frame = gray_frame
            frame_count += 1
            continue

        # Calculate frame difference to detect motion
        frame_diff = cv2.absdiff(prev_frame, gray_frame)
        motion_score = np.sum(frame_diff)

        # Debug: Log the motion score
        #print(f"Motion score: {motion_score}")

        # Check if motion is above the defined threshold and add the face to motion frames
        if motion_score > MOTION_THRESHOLD and faces:
            motion_frames.extend(faces)

        prev_frame = gray_frame
        frame_count += 1

    cap.release()
    return motion_frames, all_faces

def select_well_distributed_frames(motion_frames, all_faces, no_of_frames):
    """
    Selects well-distributed frames from the detected motion and fallback faces.

    Parameters:
    - motion_frames (list): List of frames with detected motion.
    - all_faces (list): List of all detected faces.
    - no_of_frames (int): Required number of frames.

    Returns:
    - final_frames (list): List of selected frames.
    """
    # Case 1: Motion frames exceed the required number
    if len(motion_frames) >= no_of_frames:
        interval = len(motion_frames) // no_of_frames
        distributed_motion_frames = [motion_frames[i * interval] for i in range(no_of_frames)]
        return distributed_motion_frames

    # Case 2: Motion frames are less than the required number
    needed_frames = no_of_frames - len(motion_frames)

    # If all frames together are still less than needed, return all frames available
    if len(motion_frames) + len(all_faces) < no_of_frames:
        #print(f"Returning all available frames: {len(motion_frames) + len(all_faces)}")
        return motion_frames + all_faces

    interval = max(1, len(all_faces) // needed_frames)
    additional_faces = [all_faces[i * interval] for i in range(needed_frames)]

    combined_frames = motion_frames + additional_faces
    interval = max(1, len(combined_frames) // no_of_frames)
    final_frames = [combined_frames[i * interval] for i in range(no_of_frames)]

    return final_frames

def create_dataset(no_of_frames, train_frame_folder):
    """
    Create dataset from DFDC videos by extracting frames based on motion and saving detected faces.

    Parameters:
    - no_of_frames (int): Number of frames to save per video.
    - train_frame_folder (str): Directory containing video files and metadata.json.
    """
    fake_count = 0
    videoprocessedcount = 0

    # Load metadata for video labels
    with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
        data = json.load(file)

    list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]

    with ThreadPoolExecutor(max_workers=16) as executor:
        for vid in list_of_train_data:
            if data[vid]['label'] == "FAKE" and fake_count >= 77:
                continue
            
            video_path = os.path.join(train_frame_folder, vid)

            save_dir = 'Dataset/real' if data[vid]['label'] == 'REAL' else 'Dataset/fake'

            # Extract motion frames and all faces concurrently
            motion_frames, all_faces = process_frame(video_path, detector, FRAME_SKIP)

            # Select well-distributed frames based on the motion and fallback frames
            final_frames = select_well_distributed_frames(motion_frames, all_faces, no_of_frames)

            if data[vid]['label'] == "FAKE" and len(final_frames)!=0:
                fake_count += 1

            # Save the extracted faces
            for idx, face in enumerate(final_frames):
                filename = os.path.join(save_dir, f'{vid.split(".")[0]}_{idx}.png')
                executor.submit(cv2.imwrite, filename, face)  # Save images concurrently

            videoprocessedcount += 1
            print(f"Done processing video {videoprocessedcount}: {vid}, saved {len(final_frames)} frames")

# Example usage
no_of_frames = 10  # Number of frames to extract per video
train_frame_folder = 'deepfake-detection-challenge/train_sample_videos'  # Replace with the actual path to the training folder
create_dataset(no_of_frames, train_frame_folder)


Done processing video 1: cdaxixbosp.mp4, saved 10 frames
Done processing video 2: btiysiskpf.mp4, saved 10 frames
Done processing video 3: clihsshdkq.mp4, saved 10 frames
Done processing video 4: alvgwypubw.mp4, saved 10 frames
Done processing video 5: eqvuznuwsa.mp4, saved 10 frames
Done processing video 6: eudeqjhdfd.mp4, saved 10 frames
Done processing video 7: eeyhxisdfh.mp4, saved 10 frames
Done processing video 8: cizlkenljw.mp4, saved 10 frames
Done processing video 9: bndybcqhfr.mp4, saved 10 frames
Done processing video 10: cuzrgrbvil.mp4, saved 10 frames
Done processing video 11: atyntldecu.mp4, saved 10 frames
Done processing video 12: bggsurpgpr.mp4, saved 10 frames
Done processing video 13: eckvhdusax.mp4, saved 10 frames
Done processing video 14: dvakowbgbt.mp4, saved 10 frames
Done processing video 15: dqqtjcryjv.mp4, saved 10 frames
Done processing video 16: djvutyvaio.mp4, saved 0 frames
Done processing video 17: dzwkmcwkwl.mp4, saved 10 frames
Done processing video 18

In [16]:
import os

# Check if the directory exists
fake_images_directory = 'Dataset/fake'

if os.path.exists(fake_images_directory):
    # Count the number of image files in the directory
    fake_images_count = len([file for file in os.listdir(fake_images_directory) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))])
    print(f"The number of images in the 'fake' directory: {fake_images_count}")
else:
    print("The directory does not exist. Please check the path.")

The number of images in the 'fake' directory: 770


In [18]:
!pip install opencv-python
!pip install numpy
!pip install scikit-learn
!pip install tensorflow


In [22]:
!brew install python-setuptools



==> Fetching downloads for: python-setuptools
==> Downloading https://ghcr.io/v2/homebrew/core/python-setuptools/manifests/80.
######################################################################### 100.0%
==> Fetching python-setuptools
==> Downloading https://ghcr.io/v2/homebrew/core/python-setuptools/blobs/sha256:
######################################################################### 100.0%
==> Pouring python-setuptools--80.9.0.all.bottle.tar.gz
🍺  /opt/homebrew/Cellar/python-setuptools/80.9.0: 1,000 files, 8MB
==> Running `brew cleanup python-setuptools`...
Disable this behaviour by setting `HOMEBREW_NO_INSTALL_CLEANUP=1`.
Hide these hints with `HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> No outdated dependents to upgrade!


In [ ]:
!pip install --upgrade pip
!pip install tensorflow opencv-python scikit-learn numpy

In [ ]:
!pip install tf-nightly

In [7]:
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 MB 36.2 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 44.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.3 MB/s  0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [tensorflow]3 [tensorflow]


In [12]:
!pip install tensorflow==2.17.0
!pip install keras-nightly==3.5.0.dev2024091203

In [18]:
!pip uninstall tensorflow keras -y


Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
Found existing installation: keras 3.11.3
Uninstalling keras-3.11.3:
  Successfully uninstalled keras-3.11.3


In [19]:
!pip install tensorflow==2.17.0


  Using cached tensorflow-2.17.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (4.1 kB)
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
Using cached tensorflow-2.17.0-cp312-cp312-macosx_12_0_arm64.whl (236.3 MB)
Using cached keras-3.11.3-py3-none-any.whl (1.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tensorflow]2 [tensorflow]


In [20]:
import os
import cv2
import numpy as np
import pickle
from collections import defaultdict
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
# Constants
IMG_SIZE = (299, 299)  # Target image size for InceptionV3
MAX_FRAMES = 10        # Maximum number of frames per sequence
REAL_DIR = 'real'      # Directory containing images of real samples
FAKE_DIR = 'fake'      # Directory containing images of fake samples

def load_images_from_directory(directory, label):
    """
    Load images from the specified directory, group them by videoname, preprocess, and pad sequences.
    
    Parameters:
    - directory (str): Path to the directory containing images.
    - label (int): Label for the samples (0 for real, 1 for fake).
    
    Returns:
    - data (np.array): Array of processed sequences of images.
    - labels (np.array): Array of corresponding labels.
    """
    data = []
    labels = []
    video_frames = defaultdict(list)  # Dictionary to hold frames grouped by videoname

    # Iterate over all images in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            # Extract the videoname (before the first underscore)
            video_name = filename.split('_')[0]
            filepath = os.path.join(directory, filename)
            img = cv2.imread(filepath)

            if img is not None:
                # Resize and preprocess the image
                img = cv2.resize(img, IMG_SIZE)
                img = img_to_array(img)
                img = preprocess_input(img)  # Preprocess using InceptionV3 preprocessing
                video_frames[video_name].append(img)
    
    # Process each set of images grouped by videoname
    for frames in video_frames.values():
        # Pad with zeros if frames are less than MAX_FRAMES
        while len(frames) < MAX_FRAMES:
            frames.append(np.zeros((299, 299, 3)))  # Zero-padding for missing frames

        # Limit to MAX_FRAMES if more frames are present
        frames = frames[:MAX_FRAMES]

        data.append(frames)
        labels.append(label)
    
    return np.array(data), np.array(labels)


ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
REAL_DIR = "Dataset/real"
FAKE_DIR = "Dataset/fake"
# Load real and fake images
x_real, y_real = load_images_from_directory(REAL_DIR, label=0)  # Label 0 for real
x_fake, y_fake = load_images_from_directory(FAKE_DIR, label=1)  # Label 1 for fake

NameError: name 'img_to_array' is not defined